In [426]:
#importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer,MissingIndicator, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV,KFold,StratifiedKFold,StratifiedGroupKFold,cross_val_score,cross_val_predict,KFold
from sklearn.preprocessing import PowerTransformer,OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer,make_column_selector,ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, GradientBoostingRegressor,RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import  KNNImputer
from sklearn.svm import SVC
import lightgbm as lgb
import xgboost as xgb

import warnings; warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.preprocessing import PolynomialFeatures
from feature_engine.datetime import DatetimeFeatures
from feature_engine.outliers import Winsorizer
from category_encoders import OrdinalEncoder

from feature_engine.selection import SelectByShuffling,DropHighPSIFeatures,SelectByInformationValue
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import feature_engine
from sklearn.preprocessing import KBinsDiscretizer,LabelEncoder
from feature_engine.encoding import OrdinalEncoder,PRatioEncoder
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.encoding import RareLabelEncoder,DecisionTreeEncoder
from feature_engine.selection import DropDuplicateFeatures, DropConstantFeatures
from feature_engine.selection import (RecursiveFeatureElimination, RecursiveFeatureAddition,
    DropConstantFeatures,
    DropDuplicateFeatures,
    SmartCorrelatedSelection,DropCorrelatedFeatures
)
from feature_engine.creation import MathFeatures,RelativeFeatures
from scipy.stats import chi2_contingency
# to select the features
from sklearn.feature_selection import SelectKBest, SelectPercentile
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.encoding import StringSimilarityEncoder
from feature_engine.encoding import OneHotEncoder
from feature_engine.encoding import MeanEncoder
from sklearn import metrics
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.cluster import KMeans
from category_encoders import MEstimateEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import metrics as sk_metrics


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',50)
from sklearn.metrics import log_loss


In [261]:
#importing data
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [263]:
#dropping postal codes as these belongs to different countries and dont have any relevance 
y=train['OUTCOME']
data=pd.concat((train,test)).reset_index(drop=True)
data=data.drop(columns=['ID','OUTCOME','POSTAL_CODE'],axis=1)  

train=data[:len(train)]
test=data[len(train):]
train['target']=y

In [4678]:
#combing rare values which are present less than 1 percent in data 
def rare_label(df):    
    df['DUIS']=df['DUIS'].apply(lambda x: -999 if x>=2 else x)
    df['ANNUAL_MILEAGE']=df['ANNUAL_MILEAGE'].apply(lambda x: -999 if x in [5000.0,17000.0,2000.0,18000.0,19000.0,20000.0,21000.0] else x)
    df['SPEEDING_VIOLATIONS']=df['SPEEDING_VIOLATIONS'].apply(lambda x: -999 if x>=5 else x)
    df['PAST_ACCIDENTS']=df['PAST_ACCIDENTS'].apply(lambda x: -999 if x>=5 else x)
#    df['POSTAL_CODE']=df['POSTAL_CODE'].astype('object')
    return df
train=rare_label(train)
test=rare_label(test)


In [4344]:
# def fe1(df):
#     df['total_violations']=df['SPEEDING_VIOLATIONS']+df['DUIS']+df['PAST_ACCIDENTS']
#     df['family_members']=df['MARRIED']*2 + df['CHILDREN']
   
  
    
#     df['CREDIT_SCORE']=(df['CREDIT_SCORE']*1000).round().astype('int64')
#     df['CREDIT_SCORE_rating']=df['CREDIT_SCORE'].apply(lambda x: 3 if x>=720 else 2 if x>=690 else 1 if x>=630 else 0)
    
# #     df['PAST_ACCIDENTS_bi']=df['PAST_ACCIDENTS'].apply(lambda x: 1 if x==0 else 0)
# #     df['DUIS_bi']=df['DUIS'].apply(lambda x: 1 if x==0 else 0)
# #     df['SPEEDING_VIOLATIONS_bi']=df['SPEEDING_VIOLATIONS'].apply(lambda x: 1 if x==0 else 0)
# #     df['ANNUAL_MILEAGE_rating']=df['ANNUAL_MILEAGE'].apply(lambda x: 4 if x>12000 else 3 if x> 10000 else 2 if x>8000 else 1 if x> 5000 else 0)
    
#     df['credit_cat']=df['CREDIT_SCORE'].apply(lambda x: 3 if x>700 else 2 if x>600 else 1 if x>515 else 0)
#     df['ANNUAL_MILEAGE']=df['ANNUAL_MILEAGE'].apply(lambda x: 0 if x>13000 else 1 if x>11000 else 2 if x>9000 else 3 )
    
#     del df['SPEEDING_VIOLATIONS'],df['DUIS'],df['PAST_ACCIDENTS']
    
   
#     return df
# #data=fe1(data)
# # test=fe1(test)
# train=fe1(train)
# test=fe1(test)

In [3135]:
# data['c0']=data['CHILDREN']*data['VEHICLE_OWNERSHIP']

# data['c1']=data.groupby(['TYPE_OF_VEHICLE'])['CHILDREN'].transform('max')
# data['c2']=data.groupby(['TYPE_OF_VEHICLE'])['CHILDREN'].transform('mean')
# data['c3']=data.groupby(['TYPE_OF_VEHICLE'])['CHILDREN'].transform('min')
# data['c4']=data.groupby(['TYPE_OF_VEHICLE'])['CHILDREN'].transform('std')

# data['c5']=data.groupby(['TYPE_OF_VEHICLE'])['VEHICLE_OWNERSHIP'].transform('max')
# data['c6']=data.groupby(['TYPE_OF_VEHICLE'])['VEHICLE_OWNERSHIP'].transform('mean')
# data['c7']=data.groupby(['TYPE_OF_VEHICLE'])['VEHICLE_OWNERSHIP'].transform('min')
# data['c8']=data.groupby(['TYPE_OF_VEHICLE'])['VEHICLE_OWNERSHIP'].transform('std')

## Feature Engineering

In [3136]:
def gr(df):    
    df['VEHICLE_YEAR_INCOME']=df['VEHICLE_YEAR'].astype("str")+'_'+df['INCOME'].astype("str")
  #  df['POSTAL_CODE'].astype("str")+'_'+df['AGE'].astype("str")
    return df
train=gr(train)
test=gr(test)

In [4679]:
def grouping(df_train,df_test):
    
    cats=['TYPE_OF_VEHICLE','DRIVING_EXPERIENCE']
    f1 = df_train[cats + ['target']].groupby(cats).agg(['mean', 'count']).reset_index()
    f1.columns = cats +['f1'+' '.join(col).strip() for col in f1.columns.values][2:]
    df_train=df_train.merge(f1,on=cats,how='left')
    df_test=df_test.merge(f1,on=cats,how='left')

    f2=df_train.groupby(["VEHICLE_YEAR",'INCOME']).target.agg(['mean','count']).reset_index()
    f2.columns = ['VEHICLE_YEAR','INCOME'] + [''.join(col).strip()+ "f2" for col in f2.columns.values][2:]
    df_train=df_train.merge(f2,on=['VEHICLE_YEAR','INCOME'],how='left')
    df_test=df_test.merge(f2,on=['VEHICLE_YEAR','INCOME'],how='left')

    f3=df_train.groupby(["GENDER",'TYPE_OF_VEHICLE']).target.agg(['mean','count']).reset_index()
    f3.columns = ["GENDER",'TYPE_OF_VEHICLE'] + [''.join(col).strip()+ "f3" for col in f3.columns.values][2:]
    df_train=df_train.merge(f3,on=["GENDER",'TYPE_OF_VEHICLE'],how='left')
    df_test=df_test.merge(f3,on=["GENDER",'TYPE_OF_VEHICLE'],how='left')
    
    f4=df_train.groupby(["AGE",'EDUCATION','GENDER']).target.agg(['mean','count']).reset_index()
    f4.columns = ["AGE",'EDUCATION','GENDER'] + [''.join(col).strip()+ "f4" for col in f4.columns.values][3:]
    df_train=df_train.merge(f4,on=["AGE",'EDUCATION','GENDER'],how='left')
    df_test=df_test.merge(f4,on=["AGE",'EDUCATION','GENDER'],how='left')
    
    f5=df_train.groupby(["TYPE_OF_VEHICLE",'EDUCATION','GENDER']).target.agg(['mean','count']).reset_index()
    f5.columns = ["TYPE_OF_VEHICLE",'EDUCATION','GENDER'] + [''.join(col).strip()+ "f5" for col in f5.columns.values][3:]
    df_train=df_train.merge(f5,on=["TYPE_OF_VEHICLE",'EDUCATION','GENDER'],how='left')
    df_test=df_test.merge(f5,on=["TYPE_OF_VEHICLE",'EDUCATION','GENDER'],how='left')
    
#     f6=df_train.groupby(["TYPE_OF_VEHICLE",'EDUCATION','INCOME']).target.agg(['mean','count']).reset_index()
#     f6.columns = ["TYPE_OF_VEHICLE",'EDUCATION','INCOME'] + [''.join(col).strip()+ "f6" for col in f6.columns.values][3:]
#     df_train=df_train.merge(f6,on=["TYPE_OF_VEHICLE",'EDUCATION','INCOME'],how='left')
#     df_test=df_test.merge(f6,on=["TYPE_OF_VEHICLE",'EDUCATION','INCOME'],how='left')
    
    f7=df_train.groupby(["AGE",'DRIVING_EXPERIENCE','INCOME']).target.agg(['mean','count']).reset_index()
    f7.columns = ["AGE",'DRIVING_EXPERIENCE','INCOME'] + [''.join(col).strip()+ "f7" for col in f7.columns.values][3:]
    df_train=df_train.merge(f7,on=["AGE",'DRIVING_EXPERIENCE','INCOME'],how='left')
    df_test=df_test.merge(f7,on=["AGE",'DRIVING_EXPERIENCE','INCOME'],how='left')
    
    f8=df_train.groupby(["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION']).target.agg(['mean','count']).reset_index()
    f8.columns = ["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'] + [''.join(col).strip()+ "f8" for col in f8.columns.values][4:]
    df_train=df_train.merge(f8,on=["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'],how='left')
    df_test=df_test.merge(f8,on=["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'],how='left')
    
    
    f9=df_train.groupby(["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER']).target.agg(['mean','count']).reset_index()
    f9.columns = ["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'] + [''.join(col).strip()+ "f9" for col in f9.columns.values][4:]
    df_train=df_train.merge(f9,on=["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'],how='left')
    df_test=df_test.merge(f9,on=["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'],how='left')
    
    f10=df_train.groupby(['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS']).target.agg(['mean','count']).reset_index()
    f10.columns = ['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'] + [''.join(col).strip()+ "f10" for col in f10.columns.values][3:]
    df_train=df_train.merge(f10,on=['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'],how='left')
    df_test=df_test.merge(f10,on=['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'],how='left')
    
    f11=df_train.groupby(['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE']).target.agg(['mean','count']).reset_index()
    f11.columns = ['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'] + [''.join(col).strip()+ "f11" for col in f11.columns.values][3:]
    df_train=df_train.merge(f11,on=['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'],how='left')
    df_test=df_test.merge(f11,on=['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'],how='left')





    return df_train,df_test

# data=grouping(data)
train,test=grouping(train,test)

In [4599]:
def grouping_2(df_train,df_test):
    
    cats=['TYPE_OF_VEHICLE','DRIVING_EXPERIENCE']
    f1 = df_train[cats + ['target']].groupby(cats).agg(['var','sum',]).reset_index()
    f1.columns = cats +['f1'+' '.join(col).strip() for col in f1.columns.values][2:]
    df_train=df_train.merge(f1,on=cats,how='left')
    df_test=df_test.merge(f1,on=cats,how='left')

    f2=df_train.groupby(["VEHICLE_YEAR",'INCOME']).target.agg([ 'var','sum',]).reset_index()
    f2.columns = ['VEHICLE_YEAR','INCOME'] + [''.join(col).strip()+ "f2" for col in f2.columns.values][2:]
    df_train=df_train.merge(f2,on=['VEHICLE_YEAR','INCOME'],how='left')
    df_test=df_test.merge(f2,on=['VEHICLE_YEAR','INCOME'],how='left')

    f3=df_train.groupby(["GENDER",'TYPE_OF_VEHICLE']).target.agg([ 'var','sum',]).reset_index()
    f3.columns = ["GENDER",'TYPE_OF_VEHICLE'] + [''.join(col).strip()+ "f3" for col in f3.columns.values][2:]
    df_train=df_train.merge(f3,on=["GENDER",'TYPE_OF_VEHICLE'],how='left')
    df_test=df_test.merge(f3,on=["GENDER",'TYPE_OF_VEHICLE'],how='left')
    
    f4=df_train.groupby(["AGE",'EDUCATION','GENDER']).target.agg(['var','sum',]).reset_index()
    f4.columns = ["AGE",'EDUCATION','GENDER'] + [''.join(col).strip()+ "f4" for col in f4.columns.values][3:]
    df_train=df_train.merge(f4,on=["AGE",'EDUCATION','GENDER'],how='left')
    df_test=df_test.merge(f4,on=["AGE",'EDUCATION','GENDER'],how='left')
    
    f5=df_train.groupby(["TYPE_OF_VEHICLE",'EDUCATION','GENDER']).target.agg([ 'var','sum',]).reset_index()
    f5.columns = ["TYPE_OF_VEHICLE",'EDUCATION','GENDER'] + [''.join(col).strip()+ "f5" for col in f5.columns.values][3:]
    df_train=df_train.merge(f5,on=["TYPE_OF_VEHICLE",'EDUCATION','GENDER'],how='left')
    df_test=df_test.merge(f5,on=["TYPE_OF_VEHICLE",'EDUCATION','GENDER'],how='left')
    
#     f6=df_train.groupby(["TYPE_OF_VEHICLE",'EDUCATION','INCOME']).target.agg(['mean','count']).reset_index()
#     f6.columns = ["TYPE_OF_VEHICLE",'EDUCATION','INCOME'] + [''.join(col).strip()+ "f6" for col in f6.columns.values][3:]
#     df_train=df_train.merge(f6,on=["TYPE_OF_VEHICLE",'EDUCATION','INCOME'],how='left')
#     df_test=df_test.merge(f6,on=["TYPE_OF_VEHICLE",'EDUCATION','INCOME'],how='left')
    
    f7=df_train.groupby(["AGE",'DRIVING_EXPERIENCE','INCOME']).target.agg(['var','sum',]).reset_index()
    f7.columns = ["AGE",'DRIVING_EXPERIENCE','INCOME'] + [''.join(col).strip()+ "f7" for col in f7.columns.values][3:]
    df_train=df_train.merge(f7,on=["AGE",'DRIVING_EXPERIENCE','INCOME'],how='left')
    df_test=df_test.merge(f7,on=["AGE",'DRIVING_EXPERIENCE','INCOME'],how='left')
    
    f8=df_train.groupby(["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION']).target.agg(['var','sum',]).reset_index()
    f8.columns = ["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'] + [''.join(col).strip()+ "f8" for col in f8.columns.values][4:]
    df_train=df_train.merge(f8,on=["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'],how='left')
    df_test=df_test.merge(f8,on=["AGE",'DRIVING_EXPERIENCE','INCOME','EDUCATION'],how='left')
    
    
    f9=df_train.groupby(["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER']).target.agg([ 'var','sum',]).reset_index()
    f9.columns = ["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'] + [''.join(col).strip()+ "f9" for col in f9.columns.values][4:]
    df_train=df_train.merge(f9,on=["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'],how='left')
    df_test=df_test.merge(f9,on=["INCOME",'DRIVING_EXPERIENCE','TYPE_OF_VEHICLE','GENDER'],how='left')
    
    f10=df_train.groupby(['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS']).target.agg([ 'var','sum',]).reset_index()
    f10.columns = ['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'] + [''.join(col).strip()+ "f10" for col in f10.columns.values][3:]
    df_train=df_train.merge(f10,on=['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'],how='left')
    df_test=df_test.merge(f10,on=['SPEEDING_VIOLATIONS','DUIS','PAST_ACCIDENTS'],how='left')
    
    f11=df_train.groupby(['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE']).target.agg([ 'var','sum',]).reset_index()
    f11.columns = ['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'] + [''.join(col).strip()+ "f11" for col in f11.columns.values][3:]
    df_train=df_train.merge(f11,on=['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'],how='left')
    df_test=df_test.merge(f11,on=['SPEEDING_VIOLATIONS','PAST_ACCIDENTS','AGE'],how='left')





    return df_train,df_test

# data=grouping(data)
train,test=grouping_2(train,test)

In [3613]:
def target_encode(train, valid, col, target, kfold=5, smooth=20, gpu=False):
    """
        train:  train dataset
        valid:  validation dataset
        col:   column which will be encoded (in the example RESOURCE)
        target: target column which will be used to calculate the statistic
    """
    
    # We assume that the train dataset is shuffled
    train['kfold'] = ((train.index) % kfold)
    # We keep the original order as cudf merge will not preserve the original order
    if gpu:
        train['org_sorting'] = cupy.arange(len(train), dtype="int32")
    else:
        train['org_sorting'] = np.arange(len(train), dtype="int32")
    # We create the output column, we fill with 0
    col_name = '_'.join(col)
    train['TE_' + col_name] = 0.
    for i in range(kfold):
        ###################################
        # filter for out of fold
        # calculate the mean/counts per group category
        # calculate the global mean for the oof
        # calculate the smoothed TE
        # merge it to the original dataframe
        ###################################
        
        df_tmp = train[train['kfold']!=i]
        mn = df_tmp[target].mean()
        df_tmp = df_tmp[col + [target]].groupby(col).agg(['mean', 'count']).reset_index()
        df_tmp.columns = col + ['mean', 'count']
        df_tmp['TE_tmp'] = ((df_tmp['mean']*df_tmp['count'])+(mn*smooth)) / (df_tmp['count']+smooth)
        df_tmp_m = train[col + ['kfold', 'org_sorting', 'TE_' + col_name]].merge(df_tmp, how='left', left_on=col, right_on=col).sort_values('org_sorting')
        df_tmp_m.loc[df_tmp_m['kfold']==i, 'TE_' + col_name] = df_tmp_m.loc[df_tmp_m['kfold']==i, 'TE_tmp']
        train['TE_' + col_name] = df_tmp_m['TE_' + col_name].fillna(mn).values

    
    ###################################
    # calculate the mean/counts per group for the full training dataset
    # calculate the global mean
    # calculate the smoothed TE
    # merge it to the original dataframe
    # drop all temp columns
    ###################################    
    
    df_tmp = train[col + [target]].groupby(col).agg(['mean', 'count']).reset_index()
    mn = train[target].mean()
    df_tmp.columns = col + ['mean', 'count']
    df_tmp['TE_tmp'] = ((df_tmp['mean']*df_tmp['count'])+(mn*smooth)) / (df_tmp['count']+smooth)
    if gpu:
        valid['org_sorting'] = cupy.arange(len(valid), dtype="int32")
    else:
        valid['org_sorting'] = np.arange(len(valid), dtype="int32")
    df_tmp_m = valid[col + ['org_sorting']].merge(df_tmp, how='left', left_on=col, right_on=col).sort_values('org_sorting')
    valid['TE_' + col_name] = df_tmp_m['TE_tmp'].fillna(mn).values
    
    valid = valid.drop('org_sorting', axis=1)
    train = train.drop('kfold', axis=1)
    train = train.drop('org_sorting', axis=1)
    return(train, valid)

cats=[['TYPE_OF_VEHICLE'],['GENDER'],['AGE'],
          ['EDUCATION'],['INCOME'],['GENDER','AGE','EDUCATION'],['TYPE_OF_VEHICLE','GENDER']]



for cat in cats:
    train, test = target_encode(train,test, cat, 'target')
 


In [3765]:
def count_encode(train, valid, col, gpu=False):
    """
        train:  train dataset
        valid:  validation dataset
        col:    column which will be count encoded (in the example RESOURCE)
    """
    # We keep the original order as cudf merge will not preserve the original order
    if gpu:
        train['org_sorting'] = cupy.arange(len(train), dtype="int32")
    else:
        train['org_sorting'] = np.arange(len(train), dtype="int32")
    
    train_tmp = train[col].value_counts().reset_index()
    train_tmp.columns = [col,  'CE_' + col]
    df_tmp = train[[col, 'org_sorting']].merge(train_tmp, how='left', left_on=col, right_on=col).sort_values('org_sorting')
    train['CE_' + col] = df_tmp['CE_' + col].fillna(0).values
        
    if gpu:
        valid['org_sorting'] = cupy.arange(len(valid), dtype="int32")
    else:
        valid['org_sorting'] = np.arange(len(valid), dtype="int32")
    df_tmp = valid[[col, 'org_sorting']].merge(train_tmp, how='left', left_on=col, right_on=col).sort_values('org_sorting')
    valid['CE_' + col] = df_tmp['CE_' + col].fillna(0).values
    
    valid = valid.drop('org_sorting', axis=1)
    train = train.drop('org_sorting', axis=1)
    return(train, valid)

In [4409]:
cat_cols=[col for col in test.columns if test[col].dtype=='object']
for cat in ['GENDER',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE']:
    train, test = count_encode(train,test, cat)

In [3856]:
def feat_5(df): 
    
    df['g1']=df.groupby(['EDUCATION','INCOME'])['CREDIT_SCORE'].transform('max')
    df['g2']=df.groupby(['EDUCATION','INCOME'])['CREDIT_SCORE'].transform('mean')
    df['g3']=df.groupby(['EDUCATION','INCOME'])['CREDIT_SCORE'].transform('min')
    df['g4']=df.groupby(['EDUCATION','INCOME'])['CREDIT_SCORE'].transform('std')

    df['g5']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['CREDIT_SCORE'].transform('max')
    df['g6']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['CREDIT_SCORE'].transform('mean')
    df['g7']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['CREDIT_SCORE'].transform('min')
    df['g8']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['CREDIT_SCORE'].transform('std')

    df['g9']=df.groupby(['EDUCATION','INCOME'])['ANNUAL_MILEAGE'].transform('max')
    df['g10']=df.groupby(['EDUCATION','INCOME'])['ANNUAL_MILEAGE'].transform('mean')
    df['g11']=df.groupby(['EDUCATION','INCOME'])['ANNUAL_MILEAGE'].transform('min')
    df['g12']=df.groupby(['EDUCATION','INCOME'])['ANNUAL_MILEAGE'].transform('std')

    df['g13']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['ANNUAL_MILEAGE'].transform('max')
    df['g14']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['ANNUAL_MILEAGE'].transform('mean')
    df['g15']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['ANNUAL_MILEAGE'].transform('min')
    df['g16']=df.groupby(['DRIVING_EXPERIENCE','AGE'])['ANNUAL_MILEAGE'].transform('std')
    return df
train=feat_5(train)
test=feat_5(test)

In [3741]:
def feat_6(df):    
    a=df.groupby("AGE")['GENDER',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE'].agg(['count' ,'nunique']).reset_index()
    a.columns =  ['AGE']+['AGE' +'_'.join(x) for x in a.columns][1:]

    b=df.groupby("DRIVING_EXPERIENCE")['GENDER',
     'AGE',
     'EDUCATION',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE'].agg(['count', 'nunique']).reset_index()
    b.columns =  ['DRIVING_EXPERIENCE']+['DRIVING_EXPERIENCE' +'_'.join(x) for x in a.columns][1:]

    c=df.groupby("GENDER")['AGE',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE'].agg(['count',  'nunique']).reset_index()
    c.columns =  ['GENDER']+['GENDER' +'_'.join(x) for x in a.columns][1:]

    d=df.groupby("EDUCATION")['GENDER',
     'DRIVING_EXPERIENCE',
     'AGE',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE'].agg(['count',  'nunique']).reset_index()
    d.columns =  ['EDUCATION']+['EDUCATION' +'_'.join(x) for x in a.columns][1:]

    e=df.groupby("VEHICLE_YEAR")['GENDER',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'AGE',
     'INCOME',
     'TYPE_OF_VEHICLE'].agg(['count',  'nunique']).reset_index()
    e.columns =  ['VEHICLE_YEAR']+['VEHICLE_YEAR' +'_'.join(x) for x in a.columns][1:]



    g=df.groupby("INCOME")['GENDER',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'AGE',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE'].agg(['count',  'nunique']).reset_index()
    g.columns =  ['INCOME']+['INCOME' +'_'.join(x) for x in a.columns][1:]

    df=df.merge(a,on='AGE').merge(b,on='DRIVING_EXPERIENCE').merge(c,on='GENDER').merge(d,on='EDUCATION').merge(e,on='VEHICLE_YEAR').merge(g,on='INCOME')
    return df

train=feat_6(train)
test=feat_6(test)

In [3742]:
def feat_7(df):    
    for col in ['AGE',
     'GENDER',
     'DRIVING_EXPERIENCE',
     'EDUCATION',
     'INCOME',
     'VEHICLE_YEAR',
     'TYPE_OF_VEHICLE']:
        a=df.groupby(col)['CREDIT_SCORE','ANNUAL_MILEAGE'].agg(['mean', 'std', 'min', 'max',]).reset_index()
        a.columns =  [col]+[col +'_'.join(x) for x in a.columns][1:]
        df=df.merge(a,on=col)
    return df
train=feat_7(train)
test=feat_7(test)

In [4680]:
def feat_8(df):  
    for col in ['CREDIT_SCORE','ANNUAL_MILEAGE']:
        df[col+'1']=1/df[col]
        df[col+'2']=df[col]**2
        df[col+'3']=df[col]**3
    return df
train=feat_8(train)
test=feat_8(test)

In [4681]:
def feat_9(df):    
    col_1,col_2 ='CREDIT_SCORE','ANNUAL_MILEAGE'
    
    df[col+'1']=1/df[col]
    df[col+'2']=df[col]**2
    df[col+'3']=df[col]**3
    df[col_1+col_2 +'1']=np.sqrt(df[col_1])/df[col_2]
    df[col_1+col_2+'2']=1/(df[col_1]*df[col_2])
    df[col_1+col_2+'3']=df[col_1]/df[col_2]
    df[col_1+col_2+'4']=df[col_1]**3/df[col_2]
    df[col_1+col_2+'5']=df[col_1]**2/df[col_2]
#    df[col_1+col_2+'6']=np.sqrt(df[col_1])*np.sqrt(df[col_2])
    df[col_1+col_2+'7']=np.sqrt(df[col_1])*df[col_2]**3
    #df[col_1+col_2+'8']=df[col_1]**3*np.log(df[col_2])
    df[col_1+col_2+'a']=df[col_1]**3/df[col_2]**3
  #  df[col_1+col_2+'b']=np.sqrt(df[col_1]*df[col_2])
    df[col_1+col_2+'c']=1/(df[col_1]+df[col_2])
    df[col_1+col_2+'d']=1/(df[col_1]**3*df[col_2]**3)
    return df
train=feat_9(train)
test=feat_9(test)



In [4673]:
def grp(data):    
    feat1=data.groupby("CREDIT_SCORE")["ANNUAL_MILEAGE"].transform("mean")
    data['feat2']=data['CREDIT_SCORE'] - feat1


    feat4=data.groupby("ANNUAL_MILEAGE")["CREDIT_SCORE"].transform("std")
    data['feat4']=data['ANNUAL_MILEAGE'] - feat4
    
    feat5=data.groupby("ANNUAL_MILEAGE")["CREDIT_SCORE"].transform("mean")
    data['feat5']=data['ANNUAL_MILEAGE'] - feat5
    
    return data
train=grp(train)
test=grp(test)

In [264]:
# Categorical encoding as there is order in some features, so doing Nominal encoding for these features
def cat_encode(df):    
    df['EDUCATION']=df['EDUCATION'].apply(lambda x: 0 if x=='university' else 1 if x=='high school' else 2) 
    df['DRIVING_EXPERIENCE']=df['DRIVING_EXPERIENCE'].apply(lambda x: 3 if x=='30y' else 2 if x=='20-29' else 1 if x=='10-19' else 0) 
    df['INCOME']=df['INCOME'].apply(lambda x: 3 if x=='upper class' else 2 if x=='middle class' else 1 if x=='working class' else 0)
    
#     df['edi']=df['EDUCATION']+df['DRIVING_EXPERIENCE']+df['INCOME']
#     df['edi']=df['EDUCATION']*df['DRIVING_EXPERIENCE']*df['INCOME']
    
#     df['EDUCATION_rank']=df['EDUCATION'].rank()
#     df['DRIVING_EXPERIENCE_rank']=df['DRIVING_EXPERIENCE'].rank()
#     df['total_violations_rank']=df['total_violations'].rank()
#     df['SPEEDING_VIOLATIONS_bi_rank']=df['SPEEDING_VIOLATIONS_bi'].rank()
#     df['INCOME_rank']=df['INCOME'].rank()
#     df['INCOME_rank']=df['INCOME'].rank()
#    df['VEHICLE_YEAR']=df['VEHICLE_YEAR'].apply(lambda x: 1 if x=='after' else 0) 
    df['AGE']=df['AGE'].apply(lambda x: 0 if x=='65+' else 1 if x=='40-64' else 2 if x=='26-39' else 3)

    return df

# train=cat_encode(train)
# test=cat_encode(test)
del train['target']
encoder=OrdinalEncoder()


data=pd.concat((train,test)).reset_index(drop=True)
# train=encoder.fit_transform(train,y)
# test=encoder.transform(test)
cat_cols=test.select_dtypes(include=['object'])

# for other categorical features, doing simple numeric representation of feature by identifying distinct values.
for col in cat_cols:
    data[col],_ = data[col].factorize()

train=data[:len(train)]
test=data[len(train):]
# train=pd.get_dummies(train)
# test=pd.get_dummies(test)


In [188]:

#hyperparamater tunning done by optuna
lgb_params= {'objective':'binary',
        'metric':'binary_logloss',
            'n_estimators':10000,
             'learning_rate': 0.07268222670380756,
    'boosting_type': 'gbdt',
    'lambda_l1': 0.0009158680769923057,
    'lambda_l2':  0.0298690031084307,
    'num_leaves': 109,
    'feature_fraction':0.9884585190307693,
    'bagging_fraction': 0.810897843150918,
    'bagging_freq': 4,
    'min_child_samples': 42,
    'max_bin': 307} 


lgb_model = lgb.LGBMClassifier( **lgb_params)
y_= np.log(y+1)

kf=StratifiedKFold(n_splits=5)


lr_test_a = np.zeros((len(test),5))
train_scores =[]
val_score =[]
out_of_fold = []


for fold_idx, (train_index, test_index) in enumerate(kf.split(train, y)):
    X_train,X_test,y_train,y_test= train[train_index] ,train[test_index],y[train_index], y[test_index]
    
    

    
#     calibrated_clf.fit(X_train, y_train, verbose=False,
#         eval_set=(X_test, y_test), early_stopping_rounds=500,callbacks=[lgb.log_evaluation(period=50, show_stdv=True)])
    
#     feature_importance_values += lgb_model.feature_importances_ / 5

    lgb_model.fit(X_train, y_train, verbose=False,
        eval_set=(X_test, y_test), early_stopping_rounds=500)
    
    pred_1=lgb_model.predict_proba(X_train,
                             num_iteration=lgb_model.best_iteration_)[:,1]
    pred_2=lgb_model.predict_proba(X_test,
                             num_iteration=lgb_model.best_iteration_)[:,1]
    
    

    

    lr_test_a[:,fold_idx] = lgb_model.predict_proba(test,
                             num_iteration=lgb_model.best_iteration_)[:,1]
    
    

   # loss_cliped=log_loss(y_test,np.clip(pred_1,0.055,0.975))
    training_loss=log_loss(y_train,pred_1)
    val_loss=log_loss(y_test,pred_2)
    
    #del pred_1,pred_2,lr_test_a
    
    print(f'fold no ----->{fold_idx+1}')
    print(f'training_log_loss-->>>{training_loss}')
    print(f'val_log_loss-->>>{val_loss}')
          
    print('*******************************************')
    print('*******************************************')
    
  #  print(f'cliped_log_loss--->>{loss_cliped}')
    train_scores.append(training_loss)
    val_score.append(val_loss)
    #oof_pred.append(oof)

print(f'train mean score  is ---->{np.mean(train_scores)}')
print(f'val mean score  is ---->{np.mean(val_score)}')  
#feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})

[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9884585190307693
[LightGBM] [Warning] lambda_l1 is set=0.0009158680769923057, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0009158680769923057
[LightGBM] [Warning] bagging_fraction is set=0.810897843150918, subsample=1.0 will be ignored. Current value: bagging_fraction=0.810897843150918
[LightGBM] [Warning] lambda_l2 is set=0.0298690031084307, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0298690031084307
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
fold no ----->1
training_log_loss-->>>0.6797734656278717
val_log_loss-->>>0.6810534040834404
*******************************************
*******************************************
[LightGBM] [Warning] feature_fraction is set=0.9884585190307693, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98

In [4521]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(train, y, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)
    
    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        "verbosity": -1,
        "boosting_type":  trial.suggest_categorical("boosting_type", ["gbdt",  "dart"]),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'max_bin':trial.suggest_int("max_bin", 50, 800),
    }




    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    loss = log_loss(valid_y, preds)
    return loss

In [14]:

if __name__ == "__main__":
    study = optuna.create_study(direction="minimize",sampler=sampler,
                    pruner=optuna.pruners.HyperbandPruner(min_resource=1, max_resource=n_train_iter, reduction_factor=3))
    study.optimize(objective, n_trials=100)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [251]:
a=lr_test_a.mean(axis=1)

In [4708]:
submission = pd.DataFrame({"OUTCOME":a})
submission.to_csv('submission_1071.csv', index=False)